In [1]:
fp = f"./example/old_example/all.asm"

In [2]:
from sample.cfg import proc_identify
from sample.isa import Instruction
from typing import Tuple
from sample.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 00000000004004d0
.plt 00000000004004f0
__libc_start_main@plt 0000000000400510
__gmon_start__@plt 0000000000400520
abort@plt 0000000000400530
printf@plt 0000000000400540
_start 0000000000400580
__wrap_main 00000000004005b4
.annobin_abi_note.c 00000000004005bc
_dl_relocate_static_pie 00000000004005c0
call_weak_fn 00000000004005c8
deregister_tm_clones 00000000004005e0
register_tm_clones 0000000000400610
__do_global_dtors_aux 0000000000400650
frame_dummy 0000000000400690
varFunc 0000000000400698
main 00000000004006b0
_fini 0000000000400788


In [3]:
from graphviz import Digraph
from sample.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
# g.view(quiet_view=True)
# print(g.__str__())

In [4]:
from sample.cfg import CallGraph

call_graph = CallGraph(procs)
g = call_graph.draw_graph()

g.view(quiet_view=True)
# print(g.__str__())

'call_graph.gv.svg'

In [5]:
from sample.cfg import TCfg

tcfg = TCfg(call_graph)
tcfg_nodes = tcfg.nodes

for node in tcfg_nodes:
    print(node.base_proc.name, node.inst_range)


main ('4006b0', '400700')
main ('400704', '40071c')
main ('400720', '400738')
main ('40073c', '400754')
main ('400758', '400758')
main ('40075c', '400768')
main ('40076c', '400778')
main ('40077c', '400784')
main|400700#printf@plt ('400540', '40054c')
main|40071c#printf@plt ('400540', '40054c')
main|400738#printf@plt ('400540', '40054c')
main|400754#printf@plt ('400540', '40054c')
main|400758#varFunc ('400698', '4006ac')
main|400768#printf@plt ('400540', '40054c')
main|400778#printf@plt ('400540', '40054c')
